In [ ]:
import time
import numpy as np
import pandas as pd
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

In [ ]:
def sort_coo(m):
    tuples = zip(m.row, m.col, m.data)
    return sorted(tuples, key=lambda x: (x[0], x[2]),reverse=True)[:40]

In [ ]:
start = time.clock() #_________________ measure efficiency timing

src = '/media/w/1c392724-ecf3-4615-8f3c-79368ec36380/DS Projects/Kaggle/Quora/scripts/features/'
train =  pd.read_csv(src + 'df_train_spacylemmat_fullclean.csv').iloc[:, :-1]
test =  pd.read_csv(src + 'df_test_spacylemmat_fullclean.csv').iloc[:, 4:]

# lege opvullen
train.fillna(value='NULL',inplace=True)
test.fillna(value='NULL',inplace=True)

print("Original data: trainQ: {}, testQ: {}".format(train.shape, test.shape) )
end = time.clock()
print('open:',end-start)

train_qs = pd.Series(train['question1'].tolist() + train['question2'].tolist())
test_qs = pd.Series(test['question1'].tolist() + test['question2'].tolist())
all_qs = train_qs.append(test_qs)
all_qs = all_qs.tolist()


In [ ]:
count_vectorizer = CountVectorizer(binary=True, ngram_range=(1, 2) )
count_vectorizer.fit(all_qs)  #Learn vocabulary and idf, return document freq list.
print('lengt dictionary',len(count_vectorizer.vocabulary_))
freq_term_matrix = count_vectorizer.transform(all_qs)
tfidf = TfidfTransformer(norm="l2")
tf_idf_matrix = tfidf.fit_transform(freq_term_matrix)

end = time.clock()
print('clean and make freq word dict:',end-start)

In [ ]:
all_qs=pd.DataFrame(all_qs)

In [ ]:
import hdbscan

clusterer = hdbscan.HDBSCAN(min_cluster_size=2, prediction_data=True).fit(tf_idf_matrix)

In [ ]:
corr_dict = {}

for i in range(10 - 1):
    A=tf_idf_matrix[i:i+1].dot(tf_idf_matrix.T)
    Ac=A.tocoo()
    At=sort_coo(Ac)

    question = all_qs.iloc[i, 0]
    high_cors = 0
    
    for yi in range(len(At)):
        if At[yi][2] > 0.8: 
            high_cors += 1
            
    corr_dict[question] = high_cors